# [Fondo di Garanzia](https://www.fondidigaranzia.it/amministrazione-trasparente/)


https://www.fondidigaranzia.it/normativa-e-modulistica/la-prossima-seduta-del-consiglio/


### ETL
tutti i csv del FdG:
- usano la virgola sia per il decimale che per separare i campi
- seperano i record con una riga vuota

faccio l'ETL di volta in volta con notepad++
- sostituisco "," con "|"
- sostituisco \r\n\r\n con \r\n

#### comma nei csv
dal mese di novembre 2022 viene usata la virgola come separataore di campi
La virgola va in contrasto con quelle presenti in alcuni campi

## csv to parquet

### da novembre 2022

In [1]:
### converto un solo file
import os
from io import StringIO
import pandas as pd
import requests
import duckdb

os.chdir('D:\\duckdb')

file = ("2023-M12") #  <=======================================

tsv = (file+"io.csv")
url='https://www.fondidigaranzia.it/wp-content/uploads/amm_aperta/'+file+'.csv'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}

date_column = ['delibera']
s=requests.get(url, headers= headers).text
df = pd.read_csv(StringIO(s), header=0, sep=',', quotechar='"',
                  names=['impresa', 'CF','intervento','garantito','riserva',
                         'delibera','finanziamento','ATECO','anno','settore',
                         'regione','prov','comune','processo'],
                  encoding='utf-8', decimal="," , thousands="." ,
                 parse_dates=date_column, dayfirst=True) 

df['ATECO'] = df['ATECO'].astype(str)
df['anno'] = df['anno'].astype(str)
df['intervento'] = df['intervento'].astype(str)
df['CF'] = df['CF'].astype(str)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
df.to_parquet('D:\\duckdb\\files\\FdG\\FdG_'+file+'.parquet')
df.shape

(12616, 14)

## Parquet con tutto il FdG

In [2]:
import duckdb
import os
os.chdir('D:')
conn = duckdb.connect(database='FdG.duckdb', read_only=False)
conn.execute(f'''CREATE TABLE FdG AS SELECT * FROM read_parquet('D:/duckdb/files/FdG/*.parquet');''').df()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,3140111


## [binning](https://pbpython.com/pandas-qcut-cut.html)


In [ ]:
df.columns

In [4]:
df = conn.execute("""select riserva , count(distinct(CF)) CF from fdg  group by 1 order by 2 desc;""").df()
print(df.to_string())

                                                                                                 riserva      CF
0                                                                                      Fondo di Garanzia  131423
1                                                 Fondo di Garanzia / Sezione speciale Sicilia 2014-2020    3400
2                                                   Fondo di Garanzia - Sezione Speciale Regione Toscana    2979
3                                                                                   Contributi Volontari    2874
4                                                         Fondo di Garanzia - Sez. Speciale Resto al Sud    2725
5                                                     Sezione Speciale Turismo DL 6 novembre 2021, n.152     889
6                                    Fondo di Garanzia - Sezione Speciale Emilia Romagna L.R. n. 14/2019     738
7                                                Fondo di Garanzia / Sezione speciale Campania 2

In [5]:
df = conn.execute("""select * from fdg where year(delibera)>2022;""").df()
pd.cut(df['finanziamento'],10).value_counts()

finanziamento
(-6999.0, 800900.0]       213854
(800900.0, 1600800.0]       6323
(1600800.0, 2400700.0]      1182
(2400700.0, 3200600.0]       547
(3200600.0, 4000500.0]       143
(4800400.0, 5600300.0]        75
(5600300.0, 6400200.0]        62
(4000500.0, 4800400.0]        43
(6400200.0, 7200100.0]         9
(7200100.0, 8000000.0]         4
Name: count, dtype: int64

In [6]:
df['finanziamento'].describe()

count    222242.00
mean     194757.00
std      341534.95
min        1000.00
25%       40000.00
50%      100000.00
75%      200000.00
max     8000000.00
Name: finanziamento, dtype: float64

In [9]:
df1 = pd.qcut(df['finanziamento'], q=10)

In [10]:
df1['quantile_ex_1'] = pd.qcut(df['finanziamento'], q=4)
df1['quantile_ex_2'] = pd.qcut(df['finanziamento'], q=10, precision=0)

df1

0                                              (60000.0, 100000.0]
1                                             (122000.0, 188000.0]
2                                               (50000.0, 60000.0]
3                                              (60000.0, 100000.0]
4                                              (60000.0, 100000.0]
                                       ...                        
222239                                       (485000.0, 8000000.0]
222240                                       (485000.0, 8000000.0]
222241                                          (30000.0, 50000.0]
quantile_ex_1    0           (40000.0, 100000.0]
1          (10...
quantile_ex_2    0           (60000.0, 100000.0]
1          (12...
Name: finanziamento, Length: 222244, dtype: object

In [11]:
df1['quantile_ex_2'].value_counts()

finanziamento
(60000.0, 100000.0]      36524
(30000.0, 50000.0]       31663
(999.0, 20000.0]         27622
(20000.0, 30000.0]       22661
(254487.0, 485000.0]     22225
(485000.0, 8000000.0]    22224
(188000.0, 254487.0]     22221
(122000.0, 188000.0]     22196
(50000.0, 60000.0]        7755
(100000.0, 122000.0]      7151
Name: count, dtype: int64

In [18]:
df.shape

(222242, 16)

In [12]:
cut_labels_4 = ['silver', 'gold', 'platinum', 'diamond']
cut_bins = [0, 25000, 100000, 1000000,10000000]
df['classe'] = pd.cut(df['finanziamento'], bins=cut_bins, labels=cut_labels_4)


In [13]:
df['prova2'] = pd.qcut(df['finanziamento'], 
                            q=[0, .2, .4, .6, .8, 1], 
                            labels=['Very low', 'Low', 'Moderate', 'High', 'Very high'])


In [14]:
df['prova2'].value_counts(normalize =True)

prova2
Very low    0.23
Very high   0.20
High        0.20
Moderate    0.20
Low         0.18
Name: proportion, dtype: float64

# Parquet to SQLITE

In [16]:
import pandas as pd
import pyarrow.parquet as pq
import duckdb
con = duckdb.connect()
con.execute("PRAGMA threads=2") # enable automatic query parallelization
con.execute("PRAGMA enable_object_cache") # enable caching of parquet metadata

df = con.execute("SELECT * FROM 'D:/duckdb/files/FdG/*.parquet'").df()

CPU times: total: 46.9 ms
Wall time: 1.16 s


In [17]:
%%time
import os
os.chdir('D:/')
sqlite = create_engine('sqlite:///D:/FdG.sqlite')

from sqlalchemy import create_engine
import sqlite3

df.to_sql('FdG', sqlite, if_exists='replace')